<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyse from LightGBM

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import datetime
import os, re

In [2]:
from xgboost import XGBClassifier
import xgboost
from lightgbm import LGBMClassifier
import lightgbm
import joblib

In [3]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

In [4]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import make_scorer

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [5]:
import pickle

# write list, dictionary to pickle
def save_to_pickle(path, filename):
    open_file = open(path, "wb")
    pickle.dump(filename, open_file)
    open_file.close()

# read list, dictionary from pickle
def load_from_pickle(path):
    open_file = open(path, "rb")
    loaded_file = pickle.load(open_file)
    open_file.close()
    return loaded_file

In [6]:
import csv

# write list, dictionary to csv
# path = './xxx/', my_dict = filename

def save_dict_to_csv(path, my_dict):
    df = pd.DataFrame.from_dict(my_dict, orient='index') 
    df.to_csv (path, index=False, header=True)  
    
def save_list_to_csv(path, my_list):
    df = pd.DataFrame(my_list, columns=['columns'])
    df.to_csv (path, index=False, header=True) 
    
# def load_dict_from_csv(path):
#     df = pd.read_csv(path, header=None)
#     my_dict = df.to_dict()
#     return my_dict

In [7]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
    collect_list = [ps, rs, fs, roc]
    return collect_list

In [8]:
def make_df_from_estimator(estimator, num):
    df_t = pd.DataFrame.from_dict(estimator, orient='index')
    df_t.columns = [f'value{num}']
    df_t.index.name = 'parameter'
    return df_t

In [9]:
def calc_results(lgbm, model, train_scaled, val_scaled, test_scaled, train_target, val_target, test_target,
                 test_scaled1, test_scaled2, test_scaled3, test_target1, test_target2, test_target3):
# model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
    y_predict = model.predict(test_scaled)
    result_dict= {}
    result_dict['best_score'] = lgbm.best_score_ 
    result_dict['best_index'] = lgbm.best_index_
    result_dict['acc_train'] = model.score(train_scaled, train_target)
    result_dict['acc_val'] = model.score(val_scaled, val_target)
    result_dict['acc_test'] = model.score(test_scaled, test_target)
    result_dict['precision'] = precision_score(test_target, y_predict)
    result_dict['recall'] = recall_score(test_target, y_predict)
    result_dict['f1score'] = f1_score(test_target, y_predict)
    result_dict['roc'] = roc_auc_score(test_target, y_predict)
    cm = confusion_matrix(test_target, y_predict)
    result_dict['tn'] = cm[0,0]
    result_dict['fp'] = cm[0,1]
    result_dict['fn'] = cm[1,0]
    result_dict['tp'] = cm[1,1]
#     result_dict['precision_neg'] = cm[0,0] / (cm[0,0] + cm[1,0])
    #------------------------------------
    y_predict = model.predict(test_scaled1)
    result_dict['acc_test1'] = model.score(test_scaled1, test_target1)
    result_dict['precision1'] = precision_score(test_target1, y_predict)
    result_dict['recall1'] = recall_score(test_target1, y_predict)
    result_dict['f1score1'] = f1_score(test_target1, y_predict)
    result_dict['roc1'] = roc_auc_score(test_target1, y_predict)
    cm = confusion_matrix(test_target1, y_predict)
    result_dict['tn1'] = cm[0,0]
    result_dict['fp1'] = cm[0,1]
    result_dict['fn1'] = cm[1,0]
    result_dict['tp1'] = cm[1,1]
#     result_dict['precision_neg'] = cm[0,0] / (cm[0,0] + cm[1,0])
    #------------------------------------
    y_predict = model.predict(test_scaled2)
    result_dict['acc_test2'] = model.score(test_scaled2, test_target2)
    result_dict['precision2'] = precision_score(test_target2, y_predict)
    result_dict['recall2'] = recall_score(test_target2, y_predict)
    result_dict['f1score2'] = f1_score(test_target2, y_predict)
    result_dict['roc2'] = roc_auc_score(test_target2, y_predict)
    cm = confusion_matrix(test_target2, y_predict)
    result_dict['tn2'] = cm[0,0]
    result_dict['fp2'] = cm[0,1]
    result_dict['fn2'] = cm[1,0]
    result_dict['tp2'] = cm[1,1]
#     result_dict['precision_neg'] = cm[0,0] / (cm[0,0] + cm[1,0])
    #------------------------------------
    y_predict = model.predict(test_scaled3)
    result_dict['acc_test3'] = model.score(test_scaled3, test_target3)
    result_dict['precision3'] = precision_score(test_target3, y_predict)
    result_dict['recall3'] = recall_score(test_target3, y_predict)
    result_dict['f1score3'] = f1_score(test_target3, y_predict)
    result_dict['roc3'] = roc_auc_score(test_target3, y_predict)
    cm = confusion_matrix(test_target3, y_predict)
    result_dict['tn3'] = cm[0,0]
    result_dict['fp3'] = cm[0,1]
    result_dict['fn3'] = cm[1,0]
    result_dict['tp3'] = cm[1,1]
#     result_dict['precision_neg'] = cm[0,0] / (cm[0,0] + cm[1,0])
    return result_dict

In [10]:
# save parameters and columns used for the analysis
def save_parameters(iter, com_name, dt, precision, f1score, gsbs, params, new_col):
    save_to_pickle(f'./{com_name}/params_{gsbs}_{dt}_{round(precision*100):2d}%_{round(f1score*100):2d}%_ver{iter}.pkl', params)
    save_dict_to_csv(f'./{com_name}/params_{gsbs}_{dt}_{round(precision*100):2d}%_{round(f1score*100):2d}%_ver{iter}.csv', params)
    save_to_pickle(f'./{com_name}/columns_{gsbs}_{dt}_{len(new_col)}_{round(precision*100):2d}%_{round(f1score*100):2d}%_ver{iter}.pkl', new_col)
    save_list_to_csv(f'./{com_name}/columns_{gsbs}_{dt}_{len(new_col)}_{round(precision*100):2d}%_{round(f1score*100):2d}%_ver{iter}.csv', new_col)

In [11]:
def save_best_results(com_name, model, scaler, columns, df_base):
    joblib.dump(model, f'./{com_name}/best_model.pkl') # estimaor 저장
    joblib.dump(scaler, f'./{com_name}/best_scaler.pkl') # scaler 저장
    save_to_pickle(f'./{com_name}/best_model_p.pkl', model)
    save_to_pickle(f'./{com_name}/best_scaler_p.pkl', scaler)
    save_to_pickle(f'./{com_name}/best_columns.pkl', new_col)
    save_list_to_csv(f'./{com_name}/best_columns.csv', new_col)
    df_base.to_pickle(f'./{com_name}/best_result.pkl')
    return

In [12]:
import sys, os

module_path = os.path.abspath(os.path.join('..')) # 상위 폴더로 이동
if module_path not in sys.path:
    sys.path.append(module_path+"\\data\\base_data\\common_data") #  공통으로 사용하는 각종 리스트, 코드 등 
    
import common_data as cd

In [15]:
code = cd.code_all # 전체 회사 코드
code = {'005380': ['현대차', 'hyunmotor']}

In [61]:
col_inv1 = ['retail_1', 'foreigner_1', 'institution_1', 'financial_1', 'invtrust_1', 'pension_1', 
#             'privequity_1', 'bank_1', 'insurance_1', 'financeetc_1', 'corporateetc_1', 
            'privequity_1',  'insurance_1', 'corporateetc_1', # bank_1, 'financeetc_1 제외
            'foreigneretc_1']
col_inv2 = ['retail_2', 'foreigner_2', 'institution_2', 'financial_2', 'invtrust_2', 'pension_2',
#             'privequity_2', 'bank_2', 'insurance_2', 'financeetc_2', 'corporateetc_2', 
            'privequity_2', 'insurance_2', 'corporateetc_2', # bank_2, 'financeetc_2 제외
            'foreigneretc_2']
col_his1 = ['open_1', 'high_1', 'low_1', 'close_1', 'vol_1']
col_his2 = ['open_2', 'high_2', 'low_2', 'close_2', 'vol_2']
col_cr = ['weekday', 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']
col_common1 = ["dji_cr", "dji_f_cr", "dxy_cr", "ixic_f_cr", "bond_kor_10_cr", "bond_kor_2_cr", "kosdaq_cr", "kospi_cr", 
         "krw_cr", "ixic_cr", "spx_f_cr", "sox_cr", "spx_cr", "bond_usa_10_cr", "bond_usa_2_cr", "bond_usa_3m_cr", 
         "vix_cr", "wti_cr", "spsy_cr", "spny_cr", "spxhc_cr", "splrcd_cr", "splrci_cr", "splrcu_cr", "splrcs_cr",
         "splrct_cr", "splrcl_cr", "splrcm_cr", "ixbk_cr", "ixfn_cr", "ixid_cr", "ixis_cr", "ixk_cr", "ixtr_cr",
         "ixut_cr", "nbi_cr", "bkx_cr"]
col_common2 = ["dji_cr_2", "dji_f_cr_2", "dxy_cr_2", "ixic_f_cr_2", "bond_kor_10_cr_2", "bond_kor_2_cr_2", "kosdaq_cr_2", "kospi_cr_2",
         "krw_cr_2", "ixic_cr_2", "spx_f_cr_2", "sox_cr_2", "spx_cr_2", "bond_usa_10_cr_2", "bond_usa_2_cr_2", "bond_usa_3m_cr_2",
         "vix_cr_2", "wti_cr_2", "spsy_cr_2", "spny_cr_2", "spxhc_cr_2", "splrcd_cr_2", "splrci_cr_2", "splrcu_cr_2",
         "splrcs_cr_2", "splrct_cr_2", "splrcl_cr_2", "splrcm_cr_2", "ixbk_cr_2", "ixfn_cr_2", "ixid_cr_2",
         "ixis_cr_2", "ixk_cr_2", "ixtr_cr_2", "ixut_cr_2", "nbi_cr_2", "bkx_cr_2"]
col_futures = ['ixic_f_cr', 'ixic_f_cr_2', 'spx_f_cr', 'spx_f_cr_2', 'dji_f_cr', 'dji_f_cr_2',
           'wti_cr','wti_cr_2', 'dxy_cr', 'dxy_cr_2', 'bond_usa_10_cr', 'bond_usa_10_cr_2' ]
column_o = col_inv1 + col_common1 + col_his1 + col_inv2 + col_common2 + col_his2 + col_cr

col_except_futures = [item for item in column_o if item not in col_futures] # 선물 제외

new_col = col_except_futures.copy()

# bank, financeetc는 결측치가 많아서 사용하지 않음.
# df.drop(['bank_1', 'bank_2', 'financeetc_1', 'financeetc_2'], axis=1, inplace=True)   

# col_futures : futures는 당일 종료가 되지 않는 data이므로 제외

In [18]:
# 최초의 empty df 생성
df_base = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')
iter = 0

In [20]:
#### 신규 분석 데이터는 아래 것으로 사용

com_name = list(code.values())[0][1]

directory_for_ml = '../data/data_for_ml/predict/'
fname = f'df_{com_name}_combine.pkl'
f_name = directory_for_ml + fname
df_o = pd.read_pickle(f_name) 
df_o = df_o.iloc[:-1] # /predict/를 사용할 경우 마지막 prediction data 제외

In [21]:
len(df_o)

304

In [62]:
# 반복 작업시 여기서 부터 진행 (feature importance로 선정된 새로운 column으로)

df = df_o[new_col]  # 새롭게 선정된 column으로 진행

# train, val,: 8, test: 2
split_ratio = 0.8
split_n = int(len(df)*split_ratio)

test_interval = int((len(df) - split_n)/3)
data = df.iloc[:split_n, :-5]
target = df.iloc[:split_n, -5]
test_input = df.iloc[split_n:, :-5]
test_target = df.iloc[split_n:, -5]
test_input1 = df.iloc[split_n:split_n+test_interval, :-5]
test_input2 = df.iloc[split_n+test_interval: split_n+test_interval*2, :-5]
test_input3 = df.iloc[split_n+test_interval*2:, :-5]
test_target1 = df.iloc[split_n:split_n+test_interval, -5]
test_target2 = df.iloc[split_n+test_interval: split_n+test_interval*2, -5]
test_target3 = df.iloc[split_n+test_interval*2:, -5]

train_input, val_input, train_target, val_target = \
     train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

scaler = None
scaler = StandardScaler()
scaler.fit(train_input)
train_scaled = scaler.transform(train_input)
val_scaled = scaler.transform(val_input)
test_scaled = scaler.transform(test_input)
test_scaled1 = scaler.transform(test_input1)
test_scaled2 = scaler.transform(test_input2)
test_scaled3 = scaler.transform(test_input3)

In [63]:
search_space = {
    'boosting_type' : ['gbdt', 'dart',],# ['gbdt', 'dart', 'goss'],
    'learning_rate': (0.0001, 0.004, 'log-uniform'),
    'num_leaves': (3, 20),      
    'max_depth': (0, 50), #[-1], #(0, 50),
    'min_child_samples': (5, 50),
    'min_child_weight': (5, 15), # 값을 변동시 같은 값이 최적값으로 선정되더라고 precision이 틀림. 반드시 값을 바꾸면서 진행해야 함.
    'max_bin': (100, 1000), # 사용시 0.75%로 올라감 default: 255
#     'max_cat_threshold' : (1, 100),
#     'min_split_gain' : (0.001, 0.1),
    'subsample': (0.005, 1.0, 'uniform'), # == bagging_fraction
#     'subsample_freq': (1, 10), # mobis에서 적용시 값이 많이 올라감.
    'colsample_bytree': (0.005, 1.0, 'uniform'),
    'subsample_for_bin': (100000, 1200000),
    'scale_pos_weight': np.arange(0.1, 2, 0.1) ,
    'n_estimators': (700, 10000),
#     'reg_alpha' : (0, 4.0, 'uniform'), # =='lambda_l1': [0, 5], # default 0
#     'reg_lambda' : (0, 4.0, 'uniform'), # == lambda_l2': [0, 5], #default 0    
    'reg_alpha' : np.arange(0, 3, 0.1),
    'reg_lambda' : np.arange(0, 3, 0.1),
    'force_col_wise': ['true'], 
    'boost_from_average' : [True, False],
    'objective': ['binary'],
    'metric': ['binary_logloss'],
}

params_space = search_space.copy() # for Bayesian Optimization

## BayesSearchCV

In [64]:
param_grid = {
    'cv' : 3,
#     'cv' : None,
#     'scoring' : None,
#     'scoring' : 'f1',
#     'scoring' : 'roc_auc',
    'scoring' : 'precision',
#     'scoring' : 'average_precision',
#     'scoring' : 'accuracy',
    'num_col' : len(new_col),
    'iterations' : 50,  # default : 50, number of parameter settings.
             }

In [65]:
# directory가 없으면 만드는 과정
if not os.path.exists(com_name):
    os.makedirs(com_name)

iter = iter + 1

lgbm = None
lgbmBS = None
gsbs = 'bs'

lgbmBS = BayesSearchCV( estimator = lightgbm.LGBMClassifier(verbose=0, random_state=42),
                       search_spaces = params_space,
                       scoring = param_grid['scoring'],
                       cv = StratifiedKFold( n_splits=param_grid['cv'],
                                             shuffle=True,
                                             random_state=42 ),
                       n_jobs = -1, # 자동 검색 적용
                       n_iter = param_grid['iterations'],   
                       verbose = 0, refit = True, random_state = 42 
                      )

print("*** after lgbm BS ******")
lgbmBS.fit(train_scaled, train_target, eval_metric = 'logloss') 

# save model
stamp = datetime.datetime.today().isoformat() # 파일명 끝에 생성날짜 시간 추가
dt = re.sub(r'[-:T]', '', stamp[5:16])
dt = f'{dt[:4]}_{dt[4:]}'
joblib.dump(lgbmBS.best_estimator_, f'./{com_name}/estimator_{gsbs}_{dt}_v{iter}.pkl') # bayessearchcv 저장
save_to_pickle(f'./{com_name}/model_{gsbs}_{dt}_v{iter}_p.pkl', lgbmBS.best_estimator_)
joblib.dump(scaler, f'./{com_name}/scaler_{gsbs}_{dt}_v{iter}.pkl') # scaler 저장
save_to_pickle(f'./{com_name}/scaler_{gsbs}_{dt}_v{iter}_p.pkl', scaler)

df_estimator = make_df_from_estimator(lgbmBS.best_estimator_.get_params(), iter)
df_estimator.sort_index(inplace=True) # alphabet 순으로 보기 편하게

# metrics accuracy,,,, 3단계 precision 등까지. dictionary
result_dict = calc_results(lgbmBS, lgbmBS.best_estimator_, 
                           train_scaled, val_scaled, test_scaled,  
                           train_target, val_target, test_target,
                           test_scaled1, test_scaled2, test_scaled3,  
                           test_target1, test_target2, test_target3
                          )

df_grid = make_df_from_estimator(param_grid, iter) # gridcv parameter
df_grid.sort_index(inplace=True) # alphabet 순으로 보기 편하게
df_result = make_df_from_estimator(result_dict, iter)  # dict 를 df로
df_concat = pd.concat([df_grid, df_estimator, df_result])

df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)

val_test = df_concat.loc['acc_val'].iloc[0]
acc_test = df_concat.loc['acc_test'].iloc[0]
precision = df_concat.loc['precision'].iloc[0]
f1score = df_concat.loc['f1score'].iloc[0]

df_base.to_csv(f'./{com_name}/lgbm_{gsbs}_df_{dt}_{round(precision*100):2d}%_{round(f1score*100):2d}%_ver{iter}.csv')
df_base.to_pickle(f'./{com_name}/lgbm_{gsbs}_df_{dt}_{round(precision*100):2d}%_{round(f1score*100):2d}%_ver{iter}.pkl')

save_parameters(iter, com_name, dt, precision, f1score, gsbs, params_space, new_col)
print("******* No.{} BS Process is Done! ********".format(iter))
    
print("**** End of BayesSearchCV Process ****")

*** after lgbm BS ******
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

******* No.8 BS Process is Done! ********
**** End of BayesSearchCV Process ****


In [66]:
df_base.head(60)

,value1,value2,value3,value4,value5,value6,value7,value8
parameter,,,,,,,,
cv,3,3,3,3,3,3,3,3
iterations,50,50,50,50,50,50,50,50
num_col,98,98,98,98,98,98,98,98
scoring,precision,precision,precision,precision,precision,precision,precision,precision
boost_from_average,False,False,False,True,True,True,True,True
boosting_type,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt,gbdt
class_weight,None,None,None,None,None,None,None,None
colsample_bytree,0.933204,0.933204,0.005,0.167789,0.172593,0.934283,0.03547,0.172593
force_col_wise,true,true,true,true,true,true,true,true


In [42]:
# 최종 가장 좋은 결과 저장
save_best_results(com_name, lgbmBS.best_estimator_, scaler, new_col, df_base)

In [43]:
print(lgbmBS.best_estimator_)

LGBMClassifier(boost_from_average=True, colsample_bytree=0.17259258312426423,
               force_col_wise='true', learning_rate=0.00138574255293107,
               max_bin=433, max_depth=33, metric='binary_logloss',
               min_child_samples=43, min_child_weight=9, n_estimators=4041,
               num_leaves=9, objective='binary', random_state=42, reg_alpha=1.0,
               reg_lambda=2.2, scale_pos_weight=0.5,
               subsample=0.7758686085997428, subsample_for_bin=651219,
               verbose=0)


In [52]:
model = lgbmBS.best_estimator_
feature_df = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)

In [53]:
len(feature_df)

88

In [56]:
feature_df.tail(60)

,importance
bond_usa_2_cr,17.189826
ixis_cr_2,17.041662
kosdaq_cr_2,10.972858
pension_1,5.300079
splrcu_cr_2,4.265377
privequity_2,2.693476
ixic_cr,0.920858
krw_cr,0.294223
ixfn_cr,0.000000
splrcu_cr,0.000000


In [57]:
new_col = list(feature_df.index[:36]) +  ['cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']